# 用 VLM 处理图片或视频相关的多模态任务

本文以 `HuggingFaceTB/SmolVLM-Instruct` 的 4bit 量化模型为例，展示如何处理以下多模态任务：
- 视觉问题回答（VQA）：基于图片内容回答问题
- 文本识别（OCR）：提取并识别图片中的文本文字
- 视频理解：通过分析一系列的视频帧，来生成对视频的描述

通过有效地组织提示语（文本 + 视觉），你可以将该模型应用于诸多领域，如场景理解、文档分析以及动态视觉推理。

In [ ]:
# Install the requirements in Google Colab
# !pip install transformers datasets trl huggingface_hub bitsandbytes

# Authenticate to Hugging Face
from huggingface_hub import notebook_login
notebook_login()


In [2]:
import torch, PIL
from transformers import AutoProcessor, AutoModelForVision2Seq, BitsAndBytesConfig
from transformers.image_utils import load_image

device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps" if torch.backends.mps.is_available() else "cpu"
)

quantization_config = BitsAndBytesConfig(load_in_4bit=True)
model_name = "HuggingFaceTB/SmolVLM-Instruct"
model = AutoModelForVision2Seq.from_pretrained(
    model_name,
    quantization_config=quantization_config,
).to(device)
processor = AutoProcessor.from_pretrained("HuggingFaceTB/SmolVLM-Instruct")

print(processor.image_processor.size)

`low_cpu_mem_usage` was None, now default to True since model is quantized.
You shouldn't move a model that is dispatched using accelerate hooks.
Some kwargs in processor config are unused and will not have any effect: image_seq_len. 


{'longest_edge': 1536}


## 图像级输入的情况

我们先探索生成图片的文本描述，以及针对图片提问的情况。后面我们也会探索多图输入的情况。

### 1. 单副图片的文本描述生成

In [3]:
from IPython.display import Image, display

image_url1 = "https://cdn.pixabay.com/photo/2024/11/20/09/14/christmas-9210799_1280.jpg"
display(Image(url=image_url1))

image_url2 = "https://cdn.pixabay.com/photo/2024/11/23/08/18/christmas-9218404_1280.jpg"
display(Image(url=image_url2))

In [4]:
# Load  one image
image1 = load_image(image_url1)

# Create input messages
messages = [
    {
        "role": "user",
        "content": [
            {"type": "image"},
            {"type": "text", "text": "Can you describe the image?"}
        ]
    },
]

# Prepare inputs
prompt = processor.apply_chat_template(messages, add_generation_prompt=True)
inputs = processor(text=prompt, images=[image1], return_tensors="pt")
inputs = inputs.to(device)

# Generate outputs
generated_ids = model.generate(**inputs, max_new_tokens=500)
generated_texts = processor.batch_decode(
    generated_ids,
    skip_special_tokens=True,
)

print(generated_texts)


/home/duydl/Miniconda3/envs/py310/lib/python3.10/site-packages/bitsandbytes/nn/modules.py:451: UserWarning: Input type into Linear4bit is torch.float16, but bnb_4bit_compute_dtype=torch.float32 (default). This will lead to slow inference or training speed.
  warnings.warn(


['User:<image>Can you describe the image?\nAssistant: The image is a scene of a person walking in a forest. The person is wearing a coat and a cap. The person is holding the hand of another person. The person is walking on a path. The path is covered with dry leaves. The background of the image is a forest with trees.']


### 2. 对比多幅图片
模型也可以处理并对比多幅图片。我们接下来让模型总结一下两张图片的共同主题。

In [5]:

# Load images
image2 = load_image(image_url2)

# Create input messages
messages = [
    # {
    #     "role": "user",
    #     "content": [
    #         {"type": "image"},
    #         {"type": "image"},
    #         {"type": "text", "text": "Can you describe the two images?"}
    #     ]
    # },
    {
        "role": "user",
        "content": [
            {"type": "image"},
            {"type": "image"},
            {"type": "text", "text": "What event do they both represent?"}
        ]
    },
]

# Prepare inputs
prompt = processor.apply_chat_template(messages, add_generation_prompt=True)
inputs = processor(text=prompt, images=[image1, image2], return_tensors="pt")
inputs = inputs.to(device)

# Generate outputs
generated_ids = model.generate(**inputs, max_new_tokens=500)
generated_texts = processor.batch_decode(
    generated_ids,
    skip_special_tokens=True,
)

print(generated_texts)

['User:<image>What event do they both represent?\nAssistant: Christmas.']


### 🔠 文字识别（OCR）

VLM 也可以识别并提取文字，这使得它也可以用来处理诸如文档分析之类的任务。
你也可以试试使用文字更密集的图片。

In [9]:
document_image_url = "https://cdn.pixabay.com/photo/2020/11/30/19/23/christmas-5792015_960_720.png"
display(Image(url=document_image_url))

# Load the document image
document_image = load_image(document_image_url)

# Create input message for analysis
messages = [
    {
        "role": "user",
        "content": [
            {"type": "image"},
            {"type": "text", "text": "What is written?"}
        ]
    }
]

# Prepare inputs
prompt = processor.apply_chat_template(messages, add_generation_prompt=True)
inputs = processor(text=prompt, images=[document_image], return_tensors="pt")
inputs = inputs.to(device)

# Generate outputs
generated_ids = model.generate(**inputs, max_new_tokens=500)
generated_texts = processor.batch_decode(
    generated_ids,
    skip_special_tokens=True,
)

print(generated_texts)

['User:<image>What is written?\nAssistant: MERRY CHRISTMAS AND A HAPPY NEW YEAR']


## 视频级输入的情况

视觉语言模型（VLMs）可以通过提取关键帧并按时间顺序对其进行推理，从而间接处理视频。虽然视觉语言模型缺乏专用视频模型的时间建模能力，但它们仍然能够：

- 通过按顺序分析采样帧来描述动作或事件。
- 根据具有代表性的关键帧回答有关视频的问题。
- 通过组合多个帧的文本描述来总结视频内容。

我们用这个视频作为例子：

<video width="640" height="360" controls>
  <source src="https://cdn.pixabay.com/video/2023/10/28/186794-879050032_large.mp4" type="video/mp4">
  Your browser does not support the video tag.
</video>

In [ ]:
# !pip install opencv-python

In [7]:
from IPython.display import Video
import cv2
import numpy as np

def extract_frames(video_path, max_frames=50, target_size=None):
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        raise ValueError(f"Could not open video: {video_path}")
    
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    frame_indices = np.linspace(0, total_frames - 1, max_frames, dtype=int)

    frames = []
    for idx in frame_indices:
        cap.set(cv2.CAP_PROP_POS_FRAMES, idx)
        ret, frame = cap.read()
        if ret:
            frame = PIL.Image.fromarray(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
            if target_size:
                frames.append(resize_and_crop(frame, target_size))
            else:
                frames.append(frame)
    cap.release()
    return frames

def resize_and_crop(image, target_size):
    width, height = image.size
    scale = target_size / min(width, height)
    image = image.resize((int(width * scale), int(height * scale)), PIL.Image.Resampling.LANCZOS)
    left = (image.width - target_size) // 2
    top = (image.height - target_size) // 2
    return image.crop((left, top, left + target_size, top + target_size))

# Video link
video_link = "https://cdn.pixabay.com/video/2023/10/28/186794-879050032_large.mp4"

In [8]:
question = "Describe what the woman is doing."

def generate_response(model, processor, frames, question):

    image_tokens = [{"type": "image"} for _ in frames]
    messages = [
        {
            "role": "user",
            "content": [{"type": "text", "text": "Following are the frames of a video in temporal order."}, *image_tokens, {"type": "text", "text": question}]
        }
    ]
    inputs = processor(
        text=processor.apply_chat_template(messages, add_generation_prompt=True),
        images=frames,
        return_tensors="pt"
    ).to(model.device)

    outputs = model.generate(
        **inputs, max_new_tokens=100, num_beams=5, temperature=0.7, do_sample=True, use_cache=True
    )
    return processor.decode(outputs[0], skip_special_tokens=True)


# Extract frames from the video
frames = extract_frames(video_link, max_frames=15, target_size=384)

processor.image_processor.size = (384, 384)
processor.image_processor.do_resize = False
# Generate response
response = generate_response(model, processor, frames, question)

# Display the result
# print("Question:", question)
print("Response:", response)

Response: User: Following are the frames of a video in temporal order.<image>Describe what the woman is doing.
Assistant: The woman is hanging an ornament on a Christmas tree.


## 💐 完成了！

本文展示了使用视觉语言模型的多个案例，按照文中的步骤操作，你可以对视觉语言模型展开很多试验，探索其应用场景。

后续你还可以：

- 尝试视觉语言模型（VLMs）更多的用例。
- 对 GitHub 上的 PRs 进行review。
- 通过 GitHub 提出 issue 或 PR，为完善本课程材料贡献力量。

祝你学习愉快！ 🌟